# Data preprocessing

## Preparations

### options

In [1]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### modules

In [3]:
import pandas as pd  # data mangling and transforming
import numpy as np  # handling vectors and matrices
from preproc_functions import fill_missing, split_dataset  # own preprocessing functions

### Load data & fill missing

In [4]:
# load all data
df = pd.read_csv(dir_data+'household_power_consumption.txt', 
                 sep=';', header=0, low_memory=False, 
                 infer_datetime_format=True, 
                 parse_dates={'datetime':[0,1]}, index_col=['datetime'])
# mark all missing values
df.replace('?', np.nan, inplace=True)
# make dataset numeric
df = df.astype('float32')
# fill missing
fill_missing(df.values)

FileNotFoundError: [Errno 2] File b'../data/power_consumption/household_power_consumption.txt' does not exist: b'../data/power_consumption/household_power_consumption.txt'

### create additional variable

In [ ]:
# add a column for for the remainder of sub metering
values = df.values
df['sub_metering_4'] = (values[:,0] * 1000 / 60) - (values[:,4] + 
                                                    values[:,5] + 
                                                    values[:,6])

In [ ]:
# save updated dataset
df.to_csv(dir_data+'household_power_consumption.csv')

### aggregate to days

In [ ]:
# resample data to daily
daily_groups = df.resample('D')
df_daily = daily_groups.sum()

In [ ]:
# summarize
print(df_daily.shape)
# save
df_daily.to_csv(dir_data+'household_power_consumption_by_day.csv')

## Transformations

### define time windows

In [ ]:
print(str(len(df_daily)/7)+' weeks available.')

In [ ]:
# first day
df_daily.index[0].weekday()

Dataset starts with a Saturday, we want our weeks to start with Monday however.

In [ ]:
205*0.8  # 80% of data into training set

164 weeks in training set. 41 in the test set.

In [ ]:
split_day = 2+(164*7)
training_window = (2, split_day)
test_window = (split_day, -5)

In [ ]:
print(training_window)
print(test_window)

### split into training and test

In [ ]:
train, test = split_dataset(df_daily.values, 
                            trw=training_window,
                            tew=test_window)

In [ ]:
print(train.shape)  # 164 observations of matrices 7x8
print(test.shape)  # 41 observations of same dimension

### define targets (global_active_power)

In [ ]:
train_target = train[:,:,0]
test_target = test[:,:,0]

### save locally

In [ ]:
# input
np.save(dir_data+'training_set', train)
np.save(dir_data+'test_set', test)

In [ ]:
# output
np.save(dir_data+'training_target', train_target)
np.save(dir_data+'test_target', test_target)